<a href="https://colab.research.google.com/github/ghostfm3/ksks/blob/master/word2vec2cbow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np

c = np.array([[1, 0, 0, 0, 0, 0, 0]])
W = np.random.randn(7, 3)
h = np.dot(c, W)

print(h)

[[ 1.90513461 -0.17324199 -0.24359959]]


In [ ]:
import sys
sys.path.append('..')
import numpy as np

class MatMul:
  def __init__(self, W):
    self.params = [W]
    self.grads = [np.zeros_like(W)]
    self.x = None
  
  def forward(self, x):
    W, = self.params
    out = np.dot(x, W)
    self.x = x
    return out
  
  def backward(self, dout):
    W, = self.params
    dx = np.dot(dout, W.T)
    dw = np.dot(self.x.T, dout)
    self.grads[0][...] = dw
    return dx

c = np.array([[1, 0, 0, 0, 0, 0, 0]])
W = np.random.randn(7, 3)
layer = MatMul(W)
h = layer.forward(c)
print(h)

[[-1.11136224  2.18148792 -0.24138368]]


↑ニューラルネットワークの全結合層による変換

In [ ]:
def preprocess(text):
  text = text.lower()
  text = text.replace('.', ' .')
  words = text.split(' ')

  word_to_id = {}
  id_to_word = {}
  for word in words:
    if word not in word_to_id:
      new_id = len(word_to_id)
      word_to_id[word] = new_id
      id_to_word[new_id] = word 
  
  corpus = np.array([word_to_id[w] for w in words])
  return corpus, word_to_id, id_to_word

↑preprocess関数

In [1]:
import sys
sys.path.append('..')
import numpy as np
class MatMul:
  def __init__(self, W):
    self.params = [W]
    self.grads = [np.zeros_like(W)]
    self.x = None
  
  def forward(self, x):
    W, = self.params
    out = np.dot(x, W)
    self.x = x
    return out
  
  def backward(self, dout):
    W, = self.params
    dx = np.dot(dout, W.T)
    dw = np.dot(self.x.T, dout)
    self.grads[0][...] = dw
    return dx

c0 = np.array([[1, 0, 0, 0, 0, 0, 0]])
c1 = np.array([[0, 0, 1, 0, 0, 0, 0]])

W_in = np.random.randn(7, 3)
W_out = np.random.randn(3, 7)

in_layer0 = MatMul(W_in)
in_layer1 = MatMul(W_in)
out_layer = MatMul(W_out)

h0 = in_layer0.forward(c0)
h1 = in_layer1.forward(c1)
h = 0.5 * (h0 + h1)
s = out_layer.forward(h)

print(s)


[[ 0.64336338  0.33765101 -0.79724173 -0.03986984  1.09436947  1.17927678
   0.87371495]]


↑cbowモデルにおける推論処理 p105

In [3]:
import sys
sys.path.append('..')
def preprocess(text):
  text = text.lower()
  text = text.replace('.', ' .')
  words = text.split(' ')

  word_to_id = {}
  id_to_word = {}
  for word in words:
    if word not in word_to_id:
      new_id = len(word_to_id)
      word_to_id[word] = new_id
      id_to_word[new_id] = word 
  
  corpus = np.array([word_to_id[w] for w in words])
  return corpus, word_to_id, id_to_word

text = 'You say goodbye and I say hello .'
corpus, word_to_id, id_to_word = preprocess(text)
print(corpus)

print(id_to_word)

[0 1 2 3 4 1 5 6 7]
{0: 'you', 1: 'say', 2: 'goodbye', 3: 'and', 4: 'i', 5: 'hello', 6: '', 7: '.'}


↑コーパスからコンテキストとターゲットを作成する関数 P112


In [7]:
def create_context_target(corpus, window_size=1):
  target = corpus[window_size:-window_size]
  contexts = []

  for idx in range(window_size, len(corpus)-window_size):
    cs = []
    for t in range(-window_size, window_size + 1):
      if t == 0:
        continue
      cs.append(corpus[idx + t])
    contexts.append(cs)
  
  return np.array(contexts), np.array(target)

contexts, target = create_context_target(corpus, window_size=1)
print(contexts)
print(target)

[[0 2]
 [1 3]
 [2 4]
 [3 1]
 [4 5]
 [1 6]
 [5 7]]
[1 2 3 4 1 5 6]


In [ ]:
↑コーパスからコンテキストとターゲットを作成 P113